# 🚀 LCEL (LangChain Expression Language) 핸즈온

## 📚 학습 목표
1. LCEL의 기본 개념과 장점 이해하기
2. LCEL을 활용한 체인 구성 방법 학습
3. Runnable 인터페이스의 다양한 실행 방식 파악
4. LCEL의 반환 타입과 활용 방법 이해

## 🎯 LCEL이란?
LangChain의 새로운 표현 언어로, 복잡한 체인을 간단하고 직관적으로 구성할 수 있게 해주는 도구입니다.

### 💡 주요 장점
- 체인 구성이 한 줄로 단순화
- 코드 가독성 향상
- 유지보수 용이성 증가
- 타입 안정성 보장

## 🔧 환경 설정

### 1. 필요한 패키지 설치
```bash
$ pip install langchain langchain-openai pydantic
$ poetry install langchain langchain-openai pydantic
```

### 2. 🔑 OpenAI API 키 설정

In [1]:
import dotenv
import os
dotenv.load_dotenv(".env")
if not "OPENAI_API_KEY" in os.environ:
    raise Exception("OPENAI_API_KEY가 환경변수에 존재하지 않습니다.")

 ········


## 🎮 실습 1: LCEL vs 전통적인 방식 비교

### 1.1 전통적인 방식
LCEL이 등장하기 전의 복잡한 체인 구성 방식을 살펴봅니다.

In [2]:
"""
해당 셀은, 실습전 프롬프트 및 LLM ChatModel, 파서 설정을 위한 셀입니다.
"""
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# Joke(유머) 데이터를 표현하는 Pydantic 모델 
# 또는 TypeDict로도 표현 가능하다.
class Joke(BaseModel):
    setup: str = Field(description="Joke setup question")
    punchline: str = Field(description="Joke punchline answer")


# JSON 형태로 변환할 파서
parser = JsonOutputParser(pydantic_object=Joke)

# (Prompt) 프롬프트 설정
prompt = PromptTemplate(
    template="나를 웃겨줘. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},)

# (ChatModel)OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini")

# 쿼리 생성|
joke_query = "재밌는 Joke하나 해봐"

### 1.2 전통적 방식의 실행
> 💡 **정보**: LLMChain은 Langchain이 처음 등장한 2022년, LCEL은 2023년 8월에 출시

In [3]:
# LCEL이 나오기 전에 사용했던 고전적인 방식 (LLMChain이라는 개념도 있었으나, 아래와 별 차이는 없어보임)
a = prompt.invoke({"query": joke_query})
output = llm.invoke(a)
parsed_output = parser.invoke(output)

print(parsed_output)

{'setup': '왜 바나나가 의사에게 갔을까?', 'punchline': '왜냐하면 껍질이 벗겨졌기 때문이지!'}


### 1.3 LCEL 방식
LCEL을 사용하면 위의 복잡한 과정을 한 줄로 단순화할 수 있습니다.

In [4]:
# LCEL을 활용한 방식
chain = prompt | llm | parser

# 실행
result = chain.invoke({"query": joke_query})
print(result)

{'setup': '왜 물고기는 컴퓨터를 잘 못 사용할까?', 'punchline': "왜냐하면 항상 '망가져' 버리거든!"}


## 🚀 Runnable 인터페이스

커스텀 체인을 쉽게 만들기 위해 `Runnable` 프로토콜을 사용하며, **LCEL**은 `Runnable` 인터페이스를 기반으로 동작하며 다양한 실행 방식을 제공합니다.

**LangChain** 및 **LangGrpah** 컴포넌트들은 거의 `Runnable` 프로토콜을 사용합니다.  
이는 **표준적인 방식으로 실행할 수 있고, 커스텀 체인도 쉽게 만들 수 있는 인터페이스**입니다.

### 실행 방식 비교

1. **동기 실행** (`invoke`)
   - 일반적인 실행 방식
   - 결과를 한 번에 반환

2. **비동기 실행** (`ainvoke`)
   - 비동기 처리가 필요한 경우 사용
   - `async/await` 구문과 함께 사용

3. **스트리밍 실행** (`stream`)
   - 결과를 실시간으로 받아볼 수 있음
   - Generator 객체 반환

4. **비동기 스트리밍** (`astream`)
   - 비동기 스트리밍 처리가 필요한 경우 사용
   - 비동기 Generator 객체 반환


> 💡 앞서 만든 chain 및 ChatOpenAI기반 llm 또한 모두  Runnable

### Runnable의 invoke와 stream 비교 분석

In [5]:
chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "Joke setup question", "title": "Setup", "type": "string"}, "punchline": {"description": "Joke punchline answer", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```'}, template='나를 웃겨줘. \n{format_instructions}\n{query}\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10d34d730>, async_client=<openai.resources.chat.comple

In [6]:
# 동기 실행
response = chain.invoke(joke_query)
response

{'setup': '왜 물고기는 학교에 다니지 않을까요?', 'punchline': '물속에서 항상 헤엄쳐 다니니까요!'}

In [7]:
# 스트림 실행
# stream은 Generator 타입으로 반환됨 -> 따라서, For 문으로 반복 처리 가능
for message in chain.stream(joke_query):
    print(message)

{}
{'setup': ''}
{'setup': '왜'}
{'setup': '왜 자'}
{'setup': '왜 자전'}
{'setup': '왜 자전거'}
{'setup': '왜 자전거는'}
{'setup': '왜 자전거는 넘어'}
{'setup': '왜 자전거는 넘어지'}
{'setup': '왜 자전거는 넘어지지'}
{'setup': '왜 자전거는 넘어지지 않을'}
{'setup': '왜 자전거는 넘어지지 않을까요'}
{'setup': '왜 자전거는 넘어지지 않을까요?'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': ''}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴가'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴가 있기'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴가 있기 때문'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴가 있기 때문이'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴가 있기 때문이죠'}
{'setup': '왜 자전거는 넘어지지 않을까요?', 'punchline': '두 바퀴가 있기 때문이죠!'}


### RunnablePassthrough를 활용한 매개변수 처리
- 입력값을 그대로 전달하는 특수 Runnable입니다.
- Dictionary 형태의 입력을 단순화하여 직접 값을 전달할 수 있게 해줍니다.
- {"key": RunnablePassthrough()} 형태로 사용하여 입력값을 특정 키로 매핑합니다.

간단한 RAG Chain을 구성하여 테스트

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI

# 간단한 데이터베이스 역할을 하는 dict (Retriever 대체)
fake_retriever = {
    "사과": "고담씨티 사과는 Blcak",
    "바나나": "고담씨티에서는 바나나는 Blue",
    "포도": "고담씨티에서는 포도는 Red."
}

# 간단한 Retriever 역할을 하는 함수
def retrieve_answer(question):
    # 질문 속에서 fake_retriever의 키워드가 포함된 것 찾기
    for keyword in fake_retriever.keys():
        if keyword in question:  # 질문에 해당 키워드가 포함되어 있다면 반환
            return fake_retriever[keyword]    
    return "정보를 찾을 수 없습니다."


# 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "사용자 질문: {question}\n"
    "검색된 정보: {context}\n\n"
    "위 정보에 국한하여 답변해주세요."
)

이제, 위 프롬프트와 리트리버를 기반으로, <br>
RunnablePassthrough를 활용한케이스 / RunnablePassthrough를 활용 안한 케이스 나눠서 코드 가독성 확인

In [9]:
# RunnablePassthrough를 사용한 체인 구성 및 답변수행
chain = (
    {
        "question": RunnablePassthrough(),  # 입력된 질문을 그대로 전달
        "context": retrieve_answer,  # 질문에서 키워드를 추출하고 retriever에서 답변 가져오기
    }
    | prompt  # 프롬프트 적용
    | llm  # OpenAI LLM 호출
    | StrOutputParser()  # 최종적으로 문자열로 변환
)

# 체인 실행
response = chain.invoke("고담씨티에서 사과의 색깔은?")
print(response)


고담씨티의 사과 색깔은 검은색입니다.


이게 어떻게 동작하는지? 를 보기 위해, Retriever검색에 대한 프롬프트 Chain까지의 구성을 파악해본결과, 아래와 같ㄷ다.

In [13]:
(
    {
        "question": RunnablePassthrough(),  # 입력된 질문을 그대로 전달
        "context": retrieve_answer,  # 질문에서 키워드를 추출하고 retriever에서 답변 가져오기
    }
    | prompt  # 프롬프트 적용
).invoke("고담씨티 바나나의 색깔은?")

'고담씨티에서 바나나는 파란색입니다.'

> 🛠 **예제 설명**  
> 왜 RunnablePassthrough()를 사용하는가?<br>
> - RunnablePassthrough()는 입력값을 변형 없이 그대로 전달하는 가장 간결한 방식이기 때문입니다.


##### 만약, 위 예시에서 RunnablePassthrough()를 사용하지 않는다면?
- 결과는 동일, but 코드 유연성 및 직관적인 가독성이 떨어짐

In [14]:
# 체인 실행 (RunnablePassthrough 없이 직접 데이터 구성)
def invoke_chain(question):
    # 1. 리트리버 검색
    context = retrieve_answer(question)  # 질문을 기반으로 검색 실행
    
    # 2. 프롬프트 적용
    formatted_prompt = prompt.format(question=question, context=context)
    
    # LLM 실행
    response = llm.invoke(formatted_prompt)
    
    # 결과 변환
    return StrOutputParser().invoke(response)

# 실행
response = invoke_chain("고담씨티의 바나나 색깔은?")
print(response)

고담씨티의 바나나 색깔은 파란색입니다.


> 🛠 **예제 설명**  
> 체인 없이 직접 데이터 흐름 처리시, 코드가 길어지고 수동 처리 증가한다는 단점이 존재함

### `RunnableParallel` 활용 예시
- 여러 개의 `Runnable`을 **병렬 실행**할 수 있음
- 입력값을 공유하면서 **각 체인이 독립적으로 실행**
- 결과는 `{키: 실행 결과}` 형태로 반환

✅ **장점**  
- **동시에 여러 작업 수행 가능** → 속도 최적화  
- **입력값을 공유**하면서 각 체인이 **다른 연산을 수행**  
- **비효율적인 순차 실행 대신 병렬 처리**로 성능 향상  

In [15]:
from langchain_core.runnables import RunnableParallel

# 첫 번째 체인: 곱셈
chain1 = (
    PromptTemplate.from_template("{num} 곱하기 9는? 숫자만")
    | ChatOpenAI()
    | StrOutputParser()
)

# 두 번째 체인: 나눗셈
chain2 = (
    
    PromptTemplate.from_template("{num} 나누기 9는? 숫자만")
    | ChatOpenAI()
    | StrOutputParser()
)

# 병렬 실행을 위한 RunnableParallel 구성
chains = RunnableParallel(multiply=chain1, divide=chain2)

# 실행
result = chains.invoke(9)
print(result)

/var/folders/wx/zp7p1r2n0jsb8cqslb6jl29m0000gn/T/ipykernel_51202/1225716787.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  | ChatOpenAI()


{'multiply': '81', 'divide': '1'}


> 🛠 **예제 설명**  
> ✅ `chain1`: `{num} x 9` 계산 수행  
> ✅ `chain2`: `{num} / 9` 계산 수행  
> ✅ `RunnableParallel(a1=chain1, a2=chain2)` → 두 체인을 **동시에 실행**하여 결과 반환  

### RunnableLambda
- RunnableLambda는 입력 데이터를 변환하는 **커스텀 함수를 체인 내에서 실행**할 수 있음
- **입력값을 받아 가공한 후, 다음 단계로 전달하는 역할**을 함

In [16]:
from langchain_core.runnables import RunnableLambda

chain = (
    RunnableLambda(lambda x: {"num": x * x}) # 제곱
    | PromptTemplate.from_template("{num}/2 계산해봐 ")
    | llm
    | StrOutputParser()
)

# 실행 (8을 입력하면 64/2가 계산됨)
result = chain.invoke(8)
print(result)

64 ÷ 2의 계산 결과는 32입니다.


> 🛠 **예제 설명**  
> `lambda x: {"num": x * x}` → 입력값을 제곱하여 `{num: 값}` 형태로 변환

### 🎮 실습 : Runnable 인자를 활용한 감정에 따른 LLM 프롬프트 (예시)

In [17]:
""" 해당 셀은, RunnableLambda를 활용한 감정 분석 예시를 위한 RunnableLambda 함수 정의 셀입니다. """
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage

# 감정 분석 RunnableLambda
def detect_emotion(user_input):
    # LLM 객체
    emotion_llm = ChatOpenAI(model="gpt-4o-mini")

    # 감정 분석을 위한 프롬프트 메시지 구성
    messages = [
        SystemMessage(content="너는 감정 분석 전문가야. 사용자의 감정을 '긍정', '부정', '중립', '분노' 중 하나로 정확히 판단해야 해."),
        HumanMessage(content=f"다음 문장의 감정을 분석해줘.\n문장: {user_input}\n감정:")
    ]

    # LLM을 사용하여 감정 분석 수행
    emotion = emotion_llm.invoke(messages).content.strip()

    # 감정에 따라 LLM 프롬프트 조정
    if "부정" in emotion:
        return {"query": f"사용자가 슬퍼 보이네요. 따뜻한 말 한마디 해줘\n사용자 입력: {user_input}"}
    elif "긍정" in emotion:
        return {"query": f"사용자가 행복해 보이네요! 같이 기뻐해줘\n사용자 입력: {user_input}"}
    elif "분노" in emotion:
        return {"query": f"사용자보다 더 분노해서 대답해줘\n사용자 입력: {user_input}"}
    else:
        return {"query": user_input}  # 중립이면 그대로 전달

emotion_handler = RunnableLambda(detect_emotion)

##### Chain 구성

In [18]:
# ChatOpenAI 체인
response_chain = (
    emotion_handler  # 감정 분석 및 프롬프트 조정
    | PromptTemplate.from_template("{query}")  # 조정된 프롬프트 적용
    | llm  # 응답 생성
    | StrOutputParser()
)

# 실행
response1 = response_chain.invoke("오늘 너무 힘들어...")
print("부정적 입력:", response1)  # 위로하는 답변 생성

response2 = response_chain.invoke("정말 기분 좋은 하루야!")
print("긍정적 입력:", response2)  # 함께 기뻐하는 답변 생성

response3 = response_chain.invoke("날씨가 흐리네.")
print("중립적 입력:", response3)  # 일반적인 답변 생성

response4 = response_chain.invoke("아오 짜증나!")
print("분노 입력:", response4)  # 강하게 분노를 공감하며 답변 생성


부정적 입력: 힘든 하루를 보내고 있군요. 그런 날은 누구에게나 있어요. 여러분의 고통은 이해됩니다. 잠시 숨을 고르고, 자신을 위로하는 시간을 가져보세요. 괜찮습니다, 이 또한 지나갈 거예요. 당신은 혼자가 아니에요!
긍정적 입력: 정말 기분 좋은 하루라니, 나도 함께 기뻐! 😊 어떤 일이 있어서 그렇게 좋은 하루가 되었는지 나누고 싶어?
중립적 입력: 그렇군요! 흐린 날씨는 차분한 분위기를 주기도 하고, 가끔은 비가 올 수도 있어요. 이런 날에는 따뜻한 음료를 마시며 책을 읽거나 영화 보는 게 좋죠. 혹시 특별히 하고 싶은 활동이 있나요?
분노 입력: 진짜 짜증나게 하는 상황이구나! 왜 이렇게 불만이 많은 건지 이해가 안 가! 이런 것들 때문에 스트레스받는 거 진짜 싫어! 너무 화가 나서 짜증이 폭발할 것 같아! 도대체 왜 이렇게 안 되는 거야? 왜 세상이 이렇게 불공평한 거냐고!


### Runnable 메서드의 인자 구조

모든 Runnable 메서드는 동일한 인자 구조를 가집니다:

```python
def invoke(
    input: str | BaseMessage,
    config: RunnableConfig | None = None,
    **kwargs: Any
) -> Any
```

> 💡 **참고**: LangGraph에서도 동일한 input, config 인자 구조를 사용합니다!

### `RunnableConfig` 상세 설명

`RunnableConfig`는 **LangChain의 실행 환경을 제어하는 설정 객체**입니다.  
주요 구성 요소는 다음과 같습니다:

| 속성 | 설명 |
|------|------|
| `run_name` | 해당 `Runnable`에 설정된 실행 이름 (상속되지 않음) |
| `run_id` | 이 호출에 대한 고유 식별자 (하위 호출도 별도 `run_id` 할당) |
| `tags` | 이 호출 및 하위 호출에 적용할 태그 |
| `metadata` | 이 호출 및 하위 호출에 대한 메타데이터 |
| `callbacks` | 이 호출 및 하위 호출에서 사용할 콜백 함수 |
| `max_concurrency` | 병렬 실행할 최대 호출 수 (예: 배치 처리에서 사용) |
| `recursion_limit` | 재귀 호출의 최대 횟수 (`Runnable`이 `Runnable`을 반환하는 경우) |
| `configurable` | 실행 중 동적으로 변경할 수 있는 속성 값 |

> 💡 **참고**  
> - `Runnable` 객체는 기본적으로 **두 가지 입력**을 받음: `input`(텍스트) & `config`(설정)  
> - `**kwargs`는 **추가 옵션**을 전달할 때 사용  
> - **LangGraph에서도 동일한 인자 구조**를 따름!  


In [21]:
# RunnableConfig 실제 사용 예시
from langchain_core.runnables import RunnableConfig
from langchain.callbacks import StdOutCallbackHandler

# 스트리밍 콜백 핸들러 설정
stdout_handler = StdOutCallbackHandler()

# RunnableConfig 생성
config = RunnableConfig(
    callbacks=[stdout_handler],
    tags=["demo", "streaming"],
    metadata={"session_id": "test_001"}
)

prompt = PromptTemplate(
    template="나를 웃겨줘. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},)

# 해당 구조는 Lifi(RAG Stack Builder)에서 사용자 개인화 세팅에서도 사용되는 구조
chain = prompt | llm
chain.invoke(joke_query, config=config)



> Entering new RunnableSequence chain...


> Entering new PromptTemplate chain...

> Finished chain.

> Finished chain.


AIMessage(content='```json\n{\n  "setup": "왜 자전거는 넘어지지 않았을까요?",\n  "punchline": "두 바퀴가 있어서 균형을 잡았죠!"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 211, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf63db13-fd55-4169-abd5-58b816c56287-0', usage_metadata={'input_tokens': 211, 'output_tokens': 45, 'total_tokens': 256, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 🔄 다양한 반환 타입
- LangChain은 Output Type을 다양한 형태로 결과를 반환할 수 있습니다.

##### 1. AIMessage 반환

In [23]:
response = chain.invoke(joke_query)
response

AIMessage(content='```json\n{\n  "setup": "왜 컴퓨터는 항상 가득 차 있을까?",\n  "punchline": "왜냐하면 그 안에 \'버그\'가 많아서!"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 211, 'total_tokens': 255, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3007ba0-27e8-4228-88f1-692e8acf7a67-0', usage_metadata={'input_tokens': 211, 'output_tokens': 44, 'total_tokens': 255, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### 2. Pydantic 모델 인스턴스 반환
- with_structured_output(PydanticModel)을 사용하면 Pydantic 모델 인스턴스로 결과를 받을 수 있습니다.

In [28]:
(prompt | llm.with_structured_output(Joke)).invoke({"query": joke_query})

Text(answer='{"setup": "왜 코끼리는 컴퓨터를 사용할 수 없을까요?", "punchline": "왜냐하면 그가 쥐가 없어서요!"}')

3. JSON(dict) 반환
- JsonOutputParser와 함께 사용하면 JSON 형식(dict)으로 변환하여 반환합니다.


In [29]:
(prompt | llm | parser).invoke({"query": joke_query})

{'setup': '왜 컴퓨터는 바다를 좋아할까요?', 'punchline': '왜냐하면 네트워크가 좋으니까요!'}